## Imports

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, col, trim
from datetime import date
from conexao_bancodedados import write_pyspark_mysql

## Start Spark Session

In [20]:
# Inicializa uma SparkSession
spark = SparkSession.builder \
    .appName("Leitura de Arquivo de Formato Fixo") \
    .getOrCreate()

## Variables

In [21]:
ano = date.today().year

# Define as posições iniciais e finais de cada campo
positions = {"tipo_registro": (1, 2),
               "data_pregao": (3, 10),
               "codigo_bdi": (11, 12),
               "codigo_negociacao": (13, 24),
               "tipo_mercado": (25, 27),
               "nome_resumido_empresa": (28, 39),
               "especificacao_papel": (40, 49),
               "prazo_dias_mercado": (50, 52),
               "moeda_referencia": (53, 56),
               "preco_abertura_papel": (57, 69),
               "preco_maximo_papel": (70, 82),
               "preco_minimo_papel": (83, 95),
               "preco_medio_papel": (96, 108),
               "preco_ultimo_negocio": (109, 121),
               "preco_melhor_oferta_compra": (122, 134),
               "preco_melhor_oferta_venda": (135, 147),
               "numero_negocios_efetuados": (148, 152),
               "quantidade_total_titulos": (153, 170),
               "volume_total_titulos": (171, 188),
               "preco_exercicio_opcoes": (189, 201),
               "indicador_correcao_precos": (202, 202),
               "data_vencimento_opcoes": (203, 210),
               "fator_cotacao_papel": (211, 217),
               "preco_exercicio_pontos": (218, 230),
               "codigo_papel_sistema": (231, 242),
               "numero_distribuicao_papel": (243, 245)
			   }
        

## Read File

In [22]:
# Caminho do arquivo de entrada
file_path = r'D:\Projetos\Jornada_financas_pessoais\data\COTAHIST_A2025.TXT'

# Lê o arquivo de formato fixo
df = spark.read.text(file_path)

## Transformations

In [23]:
# Extrai os campos em formato fixo usando a função substring e trim
df = df.select(
    *[trim(substring("value", start, end - start + 1)).alias(field) 
      for field, (start, end) in positions.items()]
)

df = df.filter(col("tipo_registro") == "01")

In [24]:
print(df.count(), "registros lidos")

1896003 registros lidos


## Save Data Frame

In [25]:
write_pyspark_mysql(df, 'cotahist', 'bronze', 'overwrite')

[SUCESSO] DataFrame gravado na tabela 'cotahist' (mode=overwrite)!


## Stop Spark Session

In [26]:
# Encerra a SparkSession
spark.stop()